# Preâmbulo

Imports, argumentos e definição do device.

In [ ]:
 # Basic imports.
import os
import time
import numpy as np
import torch

from torch import nn
from torch import optim
from torch.nn import functional as F

from torch.utils.data import DataLoader

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

# Setting predefined arguments.
args = {
    'epoch_num': 50,        # Number of epochs.
    # 'lr': 1e-3,             # Learning rate.
    'lr': 2e-4,             # Learning rate.
    'beta': 0.5,            # Beta1 from Adam.
    'num_workers': 2,       # Number of workers on data loader.
    'batch_size': 1000,     # Mini-batch size.
    'print_freq': 1,        # Printing frequency.
    'z_dim': 100,           # Dimension of z input vector.
    'num_samples': 3,       # Number of samples to be generated in evaluation.
    # 'least_squares': True   # Least Squares optimization.
    'least_squares': False  # Least Squares optimization.
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

# Carregando o conjunto de dados

In [ ]:
# Root directory for the dataset (to be downloaded).
root = './'

# Transformations over the dataset.
data_transforms = transforms.Compose([ # MNIST transforms.
    transforms.Pad(2, fill=0),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5)),
])
# data_transforms = transforms.Compose([ # CelebA transforms.
#     # transforms.Pad(2, fill=0),
#     transforms.Resize(64),
#     transforms.CenterCrop(64),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
# ])

# Setting datasets and dataloaders.
n_channels = 1
train_set = datasets.MNIST(root,
                           train=True,
                           download=True,
                           transform=data_transforms)

# n_channels = 3
# train_set = datasets.CelebA(root,
#                             split='all',
#                             download=True,
#                             transform=data_transforms)

# Setting dataloaders.
train_loader = DataLoader(train_set,
                          args['batch_size'],
                          num_workers=args['num_workers'],
                          shuffle=True) # MNIST
# train_loader = DataLoader(train_set,
#                           args['batch_size'],
#                           num_workers=args['num_workers'],
#                           shuffle=True,
#                           drop_last=True) # CelebA

# Printing training and testing dataset sizes.
print('Size of training set: ' + str(len(train_set)) + ' samples')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Size of training set: 60000 samples


# Treinamento Adversarial Convolucional

Como desde o começo GANs foram pensadas para imagens primariamente, era de se esperar que convoluções fossem inseridas em algum momento. O artigo original das [GANs](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf), inclusive, contém testes entre arquiteturas parcialmente convolucionais e compostas apenas por camadas FC:

*   Resultados Fully Connected no CIFAR;
![FC GAN](https://www.dropbox.com/s/dbem2z7jjzodoyb/gan_fc_goodfellow.png?dl=1)
*   Resultados Convolutionais no CIFAR.
![Convolutional GAN](https://www.dropbox.com/s/z9ihvqb4a53eqvb/gan_conv_goodfellow.png?dl=1)

A arquitetura de $G$ lembra o Decoder de um VAE com camadas que partem de um vetor aleatório $z$ e geram uma amostra final sintética $x$. Já a arquitetura de $D$ lembra uma CNN tradicional para classificação de imagens, como a AlexNet, VGG, ResNet ou DenseNet que já vimos previamente no curso:

*   Arquitetura de uma Generativa $G$;
![Rede generativa.](https://www.dropbox.com/s/q6vm8czd7shozdp/GAN_G.png?dl=1)

*   Arquitetura de uma Generativa $D$;
![Rede discriminativa.](https://www.dropbox.com/s/u4zsh5bgy5hzf4c/GAN_D.png?dl=1)

Por muito tempo, porém, foi proibitivo criar GAN convolucionais com muitas camadas, pois haviam problemas sérios de convergência e instabilidade no treinamento. O artigo das [Deep Convolutional GANs](https://arxiv.org/pdf/1511.06434.pdf) mitigou a maior parte desses problemas, propondo uma arquitetura padrão, e hoje em dia é possível treinar uma Convolutional GAN com diversas camadas.

# Prática: Implementando uma GAN Convolucional

Nessa atividade implementaremos os principais elementos de uma DCGAN, incluindo sua arquitetura e procedimento de treino. Siga os passos delineados nos TO DO's das células de código para implementar a rede generativa, a rede discriminativa, os otimizadores e o procedimento de treino.

# Definindo uma inicialização de pesos customizada.

In [ ]:
# Customized weight initialization.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Definindo o Gerador $G$

In [ ]:
# Adversarial Generator.
class Generator(nn.Module):
    
    def __init__(self, input_dim=100, output_ch=1):
        
        super(Generator, self).__init__()
        
        self.input_dim = input_dim
        self.output_ch = output_ch

        '''
        TO DO: Implemente a arquitetura da rede generativa (G). Essa arquitetura
        será composta por 4 blocos convolucionais, com cada bloco sendo composto
        pelos seguintes elementos:
        1) Convolução transposta 2d com kernel_size 4, stride 2, padding 1 e
           bias=False. Esses parâmetros garantem que a convolução transposta
           duplique as dimensões espaciais do tensor de entrada;
        2) Batch normalization 2d;
        3) ReLU.

        Obs.1: O primeiro bloco convolucional deve receber self.input_dim canais
        e retornar 256 canais. O segundo e terceiro blocos vão diminuindo essa
        quantidade de canais pela metade.

        Obs.2: O último (quarto) bloco será composto apenas pela convolução
        transposta com uma ativação tangente hiperbólica e deve retornar 1 único
        canal de saída, já ele estará tentando sintetizar uma amostra do MNIST
        que é um conjunto de dados grayscale.
        '''
        self.tconv = nn.Sequential(
            
            # Bloco 1: (B, n_Z, 2, 2) -> (B, 256, 4, 4)
            nn.ConvTranspose2d(self.input_dim, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            # Bloco 2: (B, 256, 4, 4) -> (B, 128, 8, 8)
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            
            # Bloco 3: (B, 128, 8, 8) -> (B, 64, 16, 16)
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            # Bloco 4: (B, 64, 16, 16) -> (B, Ch, 32, 32)
            nn.ConvTranspose2d(64, self.output_ch, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh(),
        )
        
    def forward(self, z):
        
        '''TO DO: Implemente o forward do vetor de ruído z ao longo dessa
        arquitetura. Lembre-se de retornar a predição gerada pela rede.'''
        x = self.tconv(z)
        
        return x

# Instantiating G.
net_G = Generator(input_dim=args['z_dim'], output_ch=n_channels).to(args['device'])
net_G.apply(weights_init)

# Printing architecture.
print(net_G)

Generator(
  (tconv): Sequential(
    (0): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)


# Definindo o Discriminador $D$

In [ ]:
# Adversarial Discriminator.
class Discriminator(nn.Module):
    
    def __init__(self, input_ch=1, output_size=1, ls=False):
        
        super(Discriminator, self).__init__()
        
        self.input_ch = input_ch
        self.output_size = output_size
        self.ls = ls

        '''
        TO DO: Implemente a arquitetura da rede discriminativa (D). Essa
        arquitetura será composta por 5 blocos convolucionais. Os 4 primeiros
        blocos terão os seguintes elementos:
        1) Convolução 2d com kernel_size 4, stride 2, padding 1 e
           bias=False. Esses parâmetros garantem que a convolução divida as
           dimensões espaciais do tensor de entrada por 2;
        2) Batch normalization 2d;
        3) LeakyReLU com parâmetro de negative slope de valor 0.2.

        O último bloco deve ser composto de uma convolução que receba a imagem
        inteira (kernel size igual às dimensões espaciais do tensor nesse ponto)
        e número de canais de saída do 4º bloco e retornar um tensor com as
        dimensões (B, 1, 1, 1), ou seja, uma única predição binária de
        classificação para cada amostra do batch. Nesse bloco, adicione uma
        ativação sigmóide.

        Obs.1: O primeiro bloco convolucional deve receber self.input_ch canais
        (no caso do MNIST, 1 canal) e retornar 64 canais. Os blocos 2, 3 e 4
        devem progressivamente duplicar esse número de canais do tensor de
        entrada.
        '''
        self.conv = nn.Sequential(

            # (B, Ch, 32, 32) -> (B, 64, 16, 16)
            nn.Conv2d(self.input_ch, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            
            # (B, 64, 16, 16) -> (B, 128, 8, 8)
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            # (B, 128, 8, 8) -> (B, 256, 4, 4)
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            # (B, 256, 4, 4) -> (B, 512, 2, 2)
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            # (B, 512, 2, 2) -> (B, 1, 1, 1)
            nn.Conv2d(512, self.output_size, kernel_size=2, stride=1, bias=False),
        )
        
    def forward(self, x):
        
        '''TO DO: Implemente o forward ao longo dessa arquitetura.'''
        x = self.conv(x)
        if not self.ls:
            x = F.sigmoid(x)

        '''TO DO: Antes de retornar o tensor de saída, aplique o método
        `.squeeze()' nas dimensões 3 e 2 do tensor de saída, deixando-o apenas
        com as dimensões (B, 1).
        '''
        return x.squeeze(3).squeeze(2)

# Instantiating D.
net_D = Discriminator(input_ch=n_channels, ls=args['least_squares']).to(args['device'])

net_D.apply(weights_init)

# Printing architecture.
print(net_D)

Discriminator(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.2, inplace=True)
    (12): Conv2d(512, 1, kernel_size=(

# Definindo o otimizadores

In [ ]:
'''TO DO: Defina um otimizador passando apenas os parâmetros do modelo
generativo. O otimizador deve ser o Adam, com LR igual a args['lr'] e betas
iguais a (args['beta'], 0.999).'''
# G optimizer.
opt_G = optim.Adam(net_G.parameters(),
                   lr=args['lr'],
                   betas=(args['beta'], 0.999))


'''TO DO: Defina um otimizador passando apenas os parâmetros do modelo
discriminativo. O otimizador deve ser o Adam, com LR igual a args['lr'] e betas
iguais a (args['beta'], 0.999).'''
# D optimizer.
opt_D = optim.Adam(net_D.parameters(),
                   lr=args['lr'],
                   betas=(args['beta'], 0.999))

# Definindo a loss composta

In [ ]:
'''TO DO: Definir a loss para treinamento da GAN. A loss utilizada
tradicionalmente por GANs é a Binary Cross Entropy (BCE).
Obs.1: A BCE no Pytorch tem implementação ligeiramente diferente da Cross
Entropy multiclasse. A documentação sobre a classe da BCE é a seguinte:
<https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html>'''
# Defining adversarial loss.
if args['least_squares']:
    criterion = nn.MSELoss().to(args['device']) # LSGAN loss.
else:
    criterion = nn.BCELoss().to(args['device']) # NSGAN loss.

'''TO DO (por último): Transforme sua NSGAN numa LSGAN ao simplesmente trocar a
BCE pela MSE e remover a ativação sigmóide do final da rede discriminativa. Link
para a loss MSE no Pytorch:
<https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html>'''

# Criando funções para Treino e Teste

In [ ]:
# Training procedure.
def train(train_loader,
          net_G, net_D,
          opt_G, opt_D,
          train_loss_G, train_loss_D,
          criterion,
          epoch):
    
    tic = time.time()
    
    # Predefining ones and zeros for batches.
    y_real = torch.ones(args['batch_size'], 1).to(args['device'])
    y_fake = torch.zeros(args['batch_size'], 1).to(args['device'])

    # Setting networks to training mode.
    net_D.train()
    net_G.train()
    
    # Iterating over batches.
    for i, batch_data in enumerate(train_loader):
        
        # Obtaining images and labels for batch.
        x, labs = batch_data
        
        '''TO DO: Criar o vetor de ruído armazenado na variável `z' amostrado de
        uma distribuição gaussiana utilizando a função `torch.randn()' com a
        forma (batch_size, args['z_dim'], 2, 2) que será alimentado para a rede
        generativa, resultando numa imagem sintética.'''
        # Creating random vector z.
        z = torch.randn((x.shape[0], args['z_dim'], 2, 2)) # MNIST (B, 1, 2, 2)
        # z = torch.randn((x.shape[0], args['z_dim'], 4, 4)) # CelebA (B, 1, 4, 4)
        
        # Casting to correct device (x and z).
        x = x.to(args['device'])
        z = z.to(args['device'])
        
        ###############
        # Updating D. #
        ###############
        
        '''TO DO: Limpar os gradients do otimizador da rede discriminativa por
        meio do método `.zero_grad()'.'''
        # Clearing the gradients of D optimizer.
        opt_D.zero_grad()

        '''TO DO: Alimentar as imagens reais para a rede discriminativa e salvar
        as predições na variável `D_real'.'''
        # Forwarding real data through D.
        D_real = net_D(x) # Through D.
        
        '''TO DO: Computar a loss das predições de D para o batch de imagens
        reais em relação aos labels das imagens reais (`y_real') e armazenar na
        variável `D_real_loss'.'''
        # Computing loss for real data.
        D_real_loss = criterion(D_real, y_real)

        '''TO DO: Alimentar o vetor de ruído `z' para a rede generativa e salvar
        as imagens sintéticas geradas na variável `G_out'.'''
        # Forwarding noise through G.
        G_out = net_G(z)
        
        '''TO DO: Alimentar as imagens sintéticas para a rede discriminativa e 
        salvar as predições na variável `D_fake'.'''
        # Forwarding synthetic samples through D.
        D_fake = net_D(G_out)
        
        '''TO DO: Computar a loss das predições de D para o batch de imagens
        sintéticas em relação aos labels das imagens sintéticas (`y_fake') e
        armazenar na variável `D_fake_loss'.'''
        # Computing loss for fake data.
        D_fake_loss = criterion(D_fake, y_fake)

        '''TO DO: Integrar as losses das imagens reais e sintéticas numa loss só
        (`D_loss'). Essa integração das losses num único batch pode ser feita de
        forma bastante simples no pytorch por meio da soma.'''
        # Computing total loss for D.
        D_loss = D_real_loss + D_fake_loss
        
        '''TO DO: Computar o backpropagation de acordo com a loss integrada da
        rede discriminativa por meio do método `.backward()'.'''
        # Computing backpropagation for D.
        D_loss.backward()
        
        '''TO DO: Chamar o `.step()' no otimizador da rede discriminativa para a
        atualização dos pesos.'''
        # Taking step in D optimizer.
        opt_D.step()

        ###############
        # Updating G. #
        ###############
        
        '''TO DO: Limpar os gradients do otimizador da rede generativa por meio
        do método `.zero_grad()'.'''
        # Clearing the gradients of G optimizer.
        opt_G.zero_grad()

        '''TO DO: Alimentar o vetor de ruído `z' para G, produzindo um batch de
        imagens sintéticas e salvando-o na variável `G_out'.'''
        # Forwarding noise through G.
        G_out = net_G(z)
        
        '''TO DO: Alimentar as imagens sintéticas geradas pela rede generativa
        para a rede discriminativa, armazenando suas predições na variável
        `D_fake'.'''
        # Forwarding synthetic samples through D.
        D_fake = net_D(G_out)
        
        '''TO DO: Computar a loss das predições da rede discriminativa passando
        as predições sobre os dados sintéticos pela discriminativa, ***mas
        enganando-a e passando rótulos reais para essas imagens***. Esse é o
        ponto crucial das NSGANs, que otimizam a rede generativa passando
        rótulos trocados para amostras sintéticas e assim achando o melhor
        conjunto de parâmetros para a rede generativa de forma a enganar a rede
        discriminativa com amostras mais e mais fotorrealistas! Armazenar a loss
        gerada nesse processo na variável `G_loss'.'''
        # Computing loss for G with swapped labels (passing fake data, but 
        # labels for real data).
        G_loss = criterion(D_fake, y_real)
        
        '''TO DO: Computar os gradientes da armazenada em `G_loss' por meio do
        método `.backward()'.'''
        # Computing backpropagation for G.
        G_loss.backward()
        
        '''TO DO: Chamar o `.step()' no otimizador da rede generativa.'''
        # Taking step in G optimizer.
        opt_G.step()
        
        # Updating lists.
        train_loss_G.append(G_loss.data.item())
        train_loss_D.append(D_loss.data.item())

    toc = time.time()
    
    # Printing training epoch loss.
    print('-------------------------------------------------------------------')
    print('[epoch %d], [training time %.2f]' % (
        epoch, (toc - tic)))
    print('-------------------------------------------------------------------')
    
    if epoch % args['print_freq'] == 0:
        
        # Plotting losses.
        fig, ax = plt.subplots(1, 2, figsize=(16, 4))

        ax[0].plot(np.asarray(train_loss_G), 'r-', label='G loss')
        ax[0].legend()
        
        ax[1].plot(np.asarray(train_loss_D), 'b--', label='D loss')
        ax[1].legend()

        plt.show()
        
    return train_loss_G, train_loss_D

In [ ]:
# Evaluating procedure.
def evaluate(net_G, criterion, epoch, sample_z):
    
    # Setting networks to training mode.
    net_D.eval()
    net_G.eval()
    
    # Casting z to correct device.
    z = sample_z.to(args['device'])
    
    # Generating new samples.
    G_out = net_G(z)
    
    # Plotting.
    fig, ax = plt.subplots(args['num_samples'],
                           args['num_samples'],
                           figsize=(8, 8))
    
    for i in range(args['num_samples']):
        
        for j in range(args['num_samples']):
            
            sample = G_out[j * args['num_samples'] + i]
            
            ax[j, i].imshow(sample.detach().cpu().numpy().squeeze(), cmap=plt.get_cmap('gray')) # MNIST
            # ax[j, i].imshow(np.transpose(sample.detach().cpu().numpy().squeeze(), axes=(1, 2, 0))) # CelebA
            ax[j, i].set_yticks([])
            ax[j, i].set_xticks([])
            
    plt.show()

# Iterando sobre epochs

In [ ]:
# Lists for losses.
train_loss_G = []
train_loss_D = []

# Creating random vector z.
sample_z = torch.randn((args['num_samples'] * args['num_samples'], args['z_dim'], 2, 2)) # MNIST
# sample_z = torch.randn((args['num_samples'] * args['num_samples'], args['z_dim'], 4, 4)) # CelebA

# Iterating over epochs.
for epoch in range(1, args['epoch_num'] + 1):

    # Training function.
    train_loss_G, train_loss_D = train(train_loader,
                                       net_G, net_D,
                                       opt_G, opt_D,
                                       train_loss_G, train_loss_D,
                                       criterion,
                                       epoch)

    if epoch % args['print_freq'] == 0:
        
        # Testing function for sample generation.
        evaluate(net_G, criterion, epoch, sample_z)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Challenge
Modifique o código atual para realizar síntese de imagens no dataset CelebA. O CelebA, assim como o MNIST, também é feito disponível no torchvision com interfaces de acesso simplificado. Para padronizar os tamanhos do dataset para imagens com dimensões (64, 64), adicione um `transforms.Resize(size=(64, 64))`, remova o padding e fixe o `transforms.Normalize()` para operar em imagens RGB no data_transforms.

Será necessário realizar alterações em várias outras áreas do código para portar o código para o CelebA!